##Importing 라이브러리

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

seed_everything(42)  # 무조건 42로 세팅!!

In [2]:
!pip install xgboost

In [3]:
from xgboost import XGBClassifier

##파일 불러오기

In [4]:
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")
submission = pd.read_csv("/content/sample_submission.csv")
print(train.shape, test.shape, submission.shape)

(48199, 13) (7695, 12) (7695, 2)


##데이터 전처리 (Train + Test)

###0)불필요 열 제거

In [5]:
train.drop(columns=['click_id', 'session ID'], inplace=True)

In [6]:
test.drop(columns=['click_id', 'session ID'], inplace=True)

###1) main category 결측치 채우기

In [7]:
#main category 결측치 채우기 --> by clothing model의 앞 문자(A:1.0, B:2.0, C:3.0, P:4.0)
mask = (train["page 1 (main category)"].isna()) #& (train["page 2 (clothing model)"].notna()) &(train["country"] == 12)
# page 2 (clothing model)의 첫 번째 문자 가져와서 매핑 (A -> 1.0, B -> 2.0, C -> 3.0, P -> 4.0)
mapping = {'A': 1.0, 'B': 2.0, 'C': 3.0, 'P': 4.0}
# 변환 적용
train.loc[mask, "page 1 (main category)"] = train.loc[mask, "page 2 (clothing model)"].str[0].map(mapping)
# 변환이 적용된 데이터 개수 확인
print(train.loc[mask, ["page 1 (main category)", "page 2 (clothing model)"]].dropna().shape[0])

31


In [8]:
#main category 결측치 채우기 --> by clothing model의 앞 문자(A:1.0, B:2.0, C:3.0, P:4.0)
mask = (test["page 1 (main category)"].isna()) #& (train["page 2 (clothing model)"].notna()) &(train["country"] == 12)
# page 2 (clothing model)의 첫 번째 문자 가져와서 매핑 (A -> 1.0, B -> 2.0, C -> 3.0, P -> 4.0)
mapping = {'A': 1.0, 'B': 2.0, 'C': 3.0, 'P': 4.0}
# 변환 적용
test.loc[mask, "page 1 (main category)"] = test.loc[mask, "page 2 (clothing model)"].str[0].map(mapping)
# 변환이 적용된 데이터 개수 확인
print(test.loc[mask, ["page 1 (main category)", "page 2 (clothing model)"]].dropna().shape[0])

3


###2)시간과 관련한 데이터 추출

In [9]:
train['date'] = pd.to_datetime(train['date'])
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day
train['weekday'] = train['date'].dt.weekday
train.drop(columns=['date'], inplace=True)

In [10]:
test['date'] = pd.to_datetime(test['date'])
test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
test['weekday'] = test['date'].dt.weekday
test.drop(columns=['date'], inplace=True)

###3)colour

In [11]:
# 색상 그룹 매핑 (높은 Click 비율 → 낮은 숫자)
color_map = {
    2: 0, 3: 0, 8: 0,  # 가장 높은 Click 비율 (가장 낮은 숫자)
    4: 1, 7: 1, 14: 1,  # 두 번째 그룹
    9: 2, 10: 2, 11: 2, 13: 2,  # 세 번째 그룹
    1: 3, 5: 3, 6: 3, 12: 3  # 가장 낮은 Click 비율 (가장 높은 숫자)
}

# colour 컬럼 변환
train["colour"] = train["colour"].map(color_map)

In [12]:
test["colour"] = test["colour"].map(color_map)

###country target encoding

In [13]:
train["country_encoded"] = train.groupby("country")["Clicked"].transform("mean")

In [14]:
test["country_encoded"] = train.groupby("country")["Clicked"].transform("mean")

###5)price log변환

In [15]:
train['log_price'] = np.log1p(train['price'])

In [16]:
test['log_price'] = np.log1p(test['price'])

###6)상품별 인기 지표

In [17]:
product_ctr = train.groupby('page 2 (clothing model)')['Clicked'].mean().reset_index()
product_ctr.columns = ['page 2 (clothing model)', 'product_ctr']

train = train.merge(product_ctr, on='page 2 (clothing model)', how='left')

In [18]:
test = test.merge(product_ctr, on='page 2 (clothing model)', how='left')
# Test 데이터에서 product_ctr가 없는 경우, Train의 평균 CTR로 대체
test['product_ctr'].fillna(train['product_ctr'].mean(), inplace=True)

###7)main category & price 관계

In [19]:
# 카테고리별 평균 가격 계산
category_price_mean = train.groupby('page 1 (main category)')['price'].mean().reset_index()
category_price_mean.columns = ['page 1 (main category)', 'category_avg_price']

# 제품의 가격 대비 카테고리 평균 가격 비율 계산
train = train.merge(category_price_mean, on='page 1 (main category)', how='left')
train['category_price_ratio'] = train['price'] / train['category_avg_price']

In [20]:
# 카테고리별 평균 가격 계산
category_price_mean = test.groupby('page 1 (main category)')['price'].mean().reset_index()
category_price_mean.columns = ['page 1 (main category)', 'category_avg_price']

# 제품의 가격 대비 카테고리 평균 가격 비율 계산
test = test.merge(category_price_mean, on='page 1 (main category)', how='left')
test['category_price_ratio'] = test['price'] / test['category_avg_price']

###7)같은 모델인데 가격이 다른 경우

In [21]:
# clothing model별 평균 가격 계산
clothing_price_mean = train.groupby('page 2 (clothing model)')['price'].mean().reset_index()
clothing_price_mean.columns = ['page 2 (clothing model)', 'clothing_avg_price']

# 개별 제품의 가격과 평균 가격 차이 계산
train = train.merge(clothing_price_mean, on='page 2 (clothing model)', how='left')
train['price_anomaly'] = train['price'] - train['clothing_avg_price']
train['price_anomaly_ratio'] = train['price'] / train['clothing_avg_price']

#page 2 (clothing model) 삭제
train.drop(columns=['page 2 (clothing model)', 'price'], inplace=True)

In [22]:
# clothing model별 평균 가격 계산
clothing_price_mean = test.groupby('page 2 (clothing model)')['price'].mean().reset_index()
clothing_price_mean.columns = ['page 2 (clothing model)', 'clothing_avg_price']

# 개별 제품의 가격과 평균 가격 차이 계산
test = test.merge(clothing_price_mean, on='page 2 (clothing model)', how='left')
test['price_anomaly'] = test['price'] - test['clothing_avg_price']
test['price_anomaly_ratio'] = test['price'] / test['clothing_avg_price']

#page 2 (clothing model) 삭제
test.drop(columns=['page 2 (clothing model)', 'price'], inplace=True)

###8)중복되는 데이터 삭제

In [23]:
train.drop(columns=['country'], inplace=True)

In [24]:
test.drop(columns=['country'], inplace=True)

##학습하기
-XGboost 모델 사용하기
-하이퍼파라미터의 영향을 많이 받는 모델이기에 최적의 조합 찾기

In [25]:
X = train.drop('Clicked', axis=1)
y = train['Clicked']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(38559, 18) (9640, 18) (38559,) (9640,)


-

In [26]:
clf=XGBClassifier()
params = {'n_estimators':[50, 100, 200],
          'max_depth':[2,3,4],
          }
grid_cv = GridSearchCV(clf, param_grid=params, cv=3, n_jobs=-1)
grid_cv.fit(X_train, y_train)
print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터:
 {'max_depth': 2, 'n_estimators': 50}
최고 예측 정확도: 0.6821


In [27]:
xgb_clf = XGBClassifier(**grid_cv.best_params_)
#[0.6844-0.6839]
#xgb_clf = XGBClassifier(n_estimators=100, max_depth=4, learning_rate = 0.05)
xgb_clf.fit(X_train, y_train)

train_pred = xgb_clf.predict(X_train)
val_pred = xgb_clf.predict(X_val)

print("Train ACC : %.4f" % accuracy_score(y_train, train_pred))
print("Val ACC : %.4f" % accuracy_score(y_val, val_pred))

Train ACC : 0.6856
Val ACC : 0.6837


In [35]:
xgb_clf = XGBClassifier(n_estimators=148, max_depth=2, learning_rate = 0.2)
xgb_clf.fit(X_train, y_train)

#148[0.6874-0.6845]
train_pred = xgb_clf.predict(X_train)
val_pred = xgb_clf.predict(X_val)

print("Train ACC : %.4f" % accuracy_score(y_train, train_pred))
print("Val ACC : %.4f" % accuracy_score(y_val, val_pred))

Train ACC : 0.6874
Val ACC : 0.6845


In [36]:
feature_importance = pd.DataFrame(xgb_clf.feature_importances_.reshape((1, -1)), columns=X_train.columns, index=['feature_importance'])
feature_importance

order  page 1 (main category)    colour  location  \
feature_importance  0.132646                  0.0823  0.021635   0.00846   

                    model photography      page  year  month       day  \
feature_importance           0.013539  0.020861   0.0    0.0  0.016242   

                     weekday  country_encoded  log_price  product_ctr  \
feature_importance  0.020337         0.015327   0.040236     0.556278   

                    category_avg_price  category_price_ratio  \
feature_importance             0.00956              0.017801   

                    clothing_avg_price  price_anomaly  price_anomaly_ratio  
feature_importance            0.018126       0.013804              0.01285

##submission.csv 생성

In [39]:
X_test = test

output = xgb_clf.predict(X_test)
assert len(output) == 7695  # sanity check

In [40]:
submission['Clicked'] = output
submission.to_csv('submission.csv', index=False)